# DocFinity REST API Playground

Welcome! With this notebook you can try out the DocFinity REST API. To get started, let's make sure everything is set up to run Jupyter Notebooks:

1. Clone the repo `git clone https://github.com/uw-it-edm/docfinity-rest-api-reference`
2. Install [Python](https://www.python.org/) (v2 or v3)
3. Install required packages:

  ```bash
      pip install jupyter
      pip install requests
      pip install json
  ```

4. To start the Jupyter server run `jupyter notebook`.
5. Select the `rest-api.ipynb` notebook.

## Configure the Access Certificate for Requests

As part of your on-boarding, you should have received a certificate, use the cell below fill in your NetID as well as the paths to your local certificate and secret.

In [1]:
# Import the common packages that will be used by all requests.
import requests
import json

baseUrl = 'https://api.dev.concert.uw.edu'
netId = 'MyNetID'  # Fill in your NetID
certificate = (
  '/path/to/cert.pem',
  '/path/to/secret.key'
)

## Create a Document

We are going to start by uploading a new document to DocFinity. Notice that the request sends the file to upload as well as a file that has the metadata as JSON to use to index the document. We are going to create the document for a document type called `Sample for REST API`, this is document type created specifically for testing so users can try out the API. However, you can update the code in the cell to target a different document type (as long as your access certificate has permissions).

The response will contain the document identifier that you can use to reference the new document and will be used in subsequent requests.


In [ ]:
url = baseUrl + '/documents/v1/create'
headers = { 'x-audituser': netId }
metadata = {
  'category': 'EDM Team',
  'documentType': 'Sample for REST API',
  'metadata': [{
      'name': 'Document Title',
      'values': ['From REST Playground']
  }]
}

files = [
  ('documentFile', ('testFile.txt','Sample File from REST API')),
  ('metadataFile',('metadata-file.json', json.dumps(metadata), 'application/json'))
]

response = requests.post(url, data={}, files=files, cert=certificate, headers=headers)
body = json.loads(response.text)
documentId = body['id']
print('Your new document id is:', body['id'])
print('To open in DocFinity go to https://uw.cloudtest.docfinity.com/docfinity/urlapi/main/viewdoc/?docid=' + documentId)

## Retrieve the Attributes for a Document

Using the document id, the request below will retrieve the attributes for the document, which are quite a lot. Notice the url is now `/docfinity` instead of `/documents`.


In [ ]:
url = baseUrl + '/docfinity/webservices/rest/document/' + documentId
headers = { 'x-audituser': netId }
response = requests.get(url, cert=certificate, headers=headers)
body = json.loads(response.text)
print(json.dumps(body, indent=2))

## Retrieve the Metadata of a Document

The request below will retrieve the list of metadata with their values. Notice that in addition to the `name` and the list of `values` there is more information included by DocFinity.

In [ ]:
url = baseUrl + '/docfinity/webservices/rest/document/details?documentId=' + documentId
headers = { 'x-audituser': netId }
response = requests.get(url, cert=certificate, headers=headers)
body = json.loads(response.text)
print(json.dumps(body, indent=2))

## Update Metadata of a Document

Finally, let's update the metadata of the document. The request below will send the list of the metadata fields to modify as JSON.

In [ ]:
url = baseUrl + '/documents/v1/update'
headers = { 
  'x-audituser': netId,
  'content-type': 'application/json'
}
metadata = {
  'id': documentId,
  'category': 'EDM Team',
  'documentType': 'Sample for REST API',
  'metadata': [
    {
      'name': 'Document Title',
      'values': ['Updated value!']
    },
    {
      'name': 'Document Description',
      'values': ['This is the new document description']
    }]
}
response = requests.post(url, data=json.dumps(metadata), cert=certificate, headers=headers)
body = json.loads(response.text)
print(json.dumps(body, indent=2))